# Summer School: Digitale Methoden der Zeitungsanalyse

## Teil 2: Alto-XML-Dateien einlesen und Text extrahieren

In diesem Teil werden zunächst die XML-Dateien eingelesen und der reine Text extrahiert. 

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Biblitoheken importiert werden.

- `pandas`: Bibliothek zur Datenanalyse.
- `lxml`: Bibliothek zur schnellen und flexiblen Verarbeitung von XML- und HTML-Dokumenten
- `BeautifulSoup`: Bibliothek zum einfachen Parsen und Scrapen von HTML- und XML-Dokumenten
- `unicodedata`: Standardbibliothek zur Handhabung und Normalisierung von Unicode-Daten
- `pathlib`: Biobliothek zur Arbeit mit Dateisystempfaden in Python
- `tqdm`: Zur Erstellung von Fortschrittsbalken in Python-Schleifen



In [2]:
import pandas as pd
from lxml import etree
from tqdm import tqdm
import unicodedata
from bs4 import BeautifulSoup
from pathlib import Path

#### Exemplarisches Laden **einer** XML-Datei: 

Direktes laden einer Datei: 

In [3]:
with open ("La_Otra_Alemania/1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DDB_FULLTEXT/5.xml") as f: 
    content = f.read()

In [4]:
print(content)

<?xml version="1.0" encoding="UTF-8"?><alto xmlns="http://www.loc.gov/standards/alto/ns-v2#" xmlns:alto="http://www.loc.gov/standards/alto/ns-v2#"><Description>
<MeasurementUnit>pixel</MeasurementUnit>
<OCRProcessing ID="IdOcr"><ocrProcessingStep><processingDateTime>2015-02-11</processingDateTime><processingSoftware><softwareCreator>ABBYY</softwareCreator><softwareName>ABBYY Recognition Server</softwareName><softwareVersion>3.0</softwareVersion></processingSoftware></ocrProcessingStep></OCRProcessing>
</Description><Styles>
<ParagraphStyle ID="StyleId-FFFFFFFF-FFFF-FFFF-FFFF-FFFFFFFFFFFF-" ALIGN="Left" LEFT="0" RIGHT="0" FIRSTLINE="0"/>
<ParagraphStyle ID="StyleId-50CB38A7-2305-4E0C-BE02-7525114F1897-" ALIGN="Block" LEFT="0" RIGHT="0" FIRSTLINE="0" LINESPACE="0"/>
<ParagraphStyle ID="StyleId-8E5FA526-EFAA-4A58-9500-AB5522F6CB0F-" ALIGN="Block" LEFT="0" RIGHT="0" FIRSTLINE="0" LINESPACE="3744"/>
<ParagraphStyle ID="StyleId-87ECC6C1-86E2-4012-AF8F-9C4DA6BDECE9-" ALIGN="Block" LEFT="0" RI

In [5]:
print(type(content))

<class 'str'>


Datei direkt als XML laden: 

In [6]:
xml = BeautifulSoup(open('La_Otra_Alemania/1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DDB_FULLTEXT/5.xml'),'lxml-xml') 
print(xml.prettify())

<?xml version="1.0" encoding="utf-8"?>
<alto:alto xmlns="http://www.loc.gov/standards/alto/ns-v2#" xmlns:alto="http://www.loc.gov/standards/alto/ns-v2#">
 <alto:Description>
  <alto:MeasurementUnit>
   pixel
  </alto:MeasurementUnit>
  <alto:OCRProcessing ID="IdOcr">
   <alto:ocrProcessingStep>
    <alto:processingDateTime>
     2015-02-11
    </alto:processingDateTime>
    <alto:processingSoftware>
     <alto:softwareCreator>
      ABBYY
     </alto:softwareCreator>
     <alto:softwareName>
      ABBYY Recognition Server
     </alto:softwareName>
     <alto:softwareVersion>
      3.0
     </alto:softwareVersion>
    </alto:processingSoftware>
   </alto:ocrProcessingStep>
  </alto:OCRProcessing>
 </alto:Description>
 <alto:Styles>
  <alto:ParagraphStyle ALIGN="Left" FIRSTLINE="0" ID="StyleId-FFFFFFFF-FFFF-FFFF-FFFF-FFFFFFFFFFFF-" LEFT="0" RIGHT="0"/>
  <alto:ParagraphStyle ALIGN="Block" FIRSTLINE="0" ID="StyleId-50CB38A7-2305-4E0C-BE02-7525114F1897-" LEFT="0" LINESPACE="0" RIGHT="0"/>


In [7]:
print(type(xml))

<class 'bs4.BeautifulSoup'>


#### Laden **aller** .xml-Dateien aus einem Unterordner

Laden aller Dateien aus dem Unterverzeichnis "La_Otra_Alemania/1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4":

In [8]:
folder_xml = []

for filepath in Path('./La_Otra_Alemania/1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4').glob('*/*.XML'):
    with filepath.open() as f:
        soup = BeautifulSoup(f,'lxml-xml')
        folder_xml.append(soup) 

In [9]:
print(len(folder_xml))

27


In [10]:
print(folder_xml)

[<?xml version="1.0" encoding="utf-8"?>
<alto:alto xmlns="http://www.loc.gov/standards/alto/ns-v2#" xmlns:alto="http://www.loc.gov/standards/alto/ns-v2#"><alto:Description>
<alto:MeasurementUnit>pixel</alto:MeasurementUnit>
<alto:OCRProcessing ID="IdOcr"><alto:ocrProcessingStep><alto:processingDateTime>2015-02-11</alto:processingDateTime><alto:processingSoftware><alto:softwareCreator>ABBYY</alto:softwareCreator><alto:softwareName>ABBYY Recognition Server</alto:softwareName><alto:softwareVersion>3.0</alto:softwareVersion></alto:processingSoftware></alto:ocrProcessingStep></alto:OCRProcessing>
</alto:Description><alto:Styles>
<alto:ParagraphStyle ALIGN="Left" FIRSTLINE="0" ID="StyleId-FFFFFFFF-FFFF-FFFF-FFFF-FFFFFFFFFFFF-" LEFT="0" RIGHT="0"/>
<alto:ParagraphStyle ALIGN="Block" FIRSTLINE="0" ID="StyleId-50CB38A7-2305-4E0C-BE02-7525114F1897-" LEFT="0" LINESPACE="0" RIGHT="0"/>
<alto:ParagraphStyle ALIGN="Block" FIRSTLINE="0" ID="StyleId-8E5FA526-EFAA-4A58-9500-AB5522F6CB0F-" LEFT="0" LINE

#### Laden ALLER .xml-Dateien im Unterverzeichnis "La_Otra_Alemania"

Statt einer Liste wird nun ein Dictionary erzeugt, welches zur besseren Nachvollziehbarkeit als Schlüssel auch den jeweiligen Dateinamen enthält:

In [11]:
all_xml = {}

for filepath in Path('./La_Otra_Alemania').glob('**/*.XML'):
    with filepath.open(encoding='utf-8') as f:
        # read as string: 
        xml_string = f.read()
        #Dateiname inklusive der Namen der beiden übergeordneten Ordner als Schlüssel:
        key = f"{filepath.parent.parent.name}/{filepath.parent.name}/{filepath.name}"
        all_xml[key] = xml_string

In [12]:
print(len(all_xml))

501


#### Kurze Erkärung zu .glob: 

- Mustererkennung: Die glob-Methode wird verwendet, um Dateipfade mit einem bestimmten Muster abzugleichen. In diesem Fall ist das Muster '\*/\*.XML'.
- Musterdetails:
    - '\*' entspricht einer beliebigen Anzahl von Zeichen, einschließlich keinem.
    - '\*.XML' entspricht jeder Datei mit der Erweiterung '.XML'.
    - Das Muster '\*/\*.XML' sucht speziell nach '.XML'-Dateien, die sich eine Verzeichnisebene unterhalb des angegebenen Pfads ('./Folder') befinden.
- Rekursive Suche: Das Muster '\*/\*.XML' sucht nach XML-Dateien in allen unmittelbaren Unterverzeichnissen von './Folder'. Um rekursiv durch alle Unterverzeichnisse in beliebiger Tiefe zu suchen, wird das Muster '\*\*/\*.XML' verwendet.


#### Umwandlung in ein Pandas-Dataframe


In [13]:
df = pd.DataFrame(list(all_xml.items()), columns=['filename', 'content'])
df

,filename,content
0,1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm..."
1,1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm..."
2,1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm..."
3,1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm..."
4,1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm..."
...,...,...
496,1943-04-01_M3RQ2MXZFORPDLNXQA57XR6C5MNN3EFS/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm..."
497,1943-04-01_M3RQ2MXZFORPDLNXQA57XR6C5MNN3EFS/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm..."
498,1943-04-01_M3RQ2MXZFORPDLNXQA57XR6C5MNN3EFS/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm..."
499,1943-04-01_M3RQ2MXZFORPDLNXQA57XR6C5MNN3EFS/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm..."


#### Suchen von möglichen Problemen (optional): 

In [28]:
# Sucht nach dem Index von Reihen, in denen die Spalte "text" keinen Inhalt hat:
empty_text_indices = df[df['text'] == ""].index

# Ausgabe der gefundenen Zeilen ohne Text: 
for index in empty_text_indices:
    print(f"Index: {index}, Content: {df.loc[index, 'Filename']}")

Index([], dtype='int64')


#### Text aus XML extrahieren: 

In [29]:
def extract_text(content):
    # Remove XML declaration if present
    if content.startswith('<?xml'):
        content = content.split('?>', 1)[1]

    NS = {'alto': 'http://www.loc.gov/standards/alto/ns-v2#'}
    tree = etree.fromstring(unicodedata.normalize("NFC", content))
    
    text_lines = []  # Initialize as an empty list to store text lines
    
    for line in tree.xpath('//alto:TextLine', namespaces=NS):
        text = " ".join(
            word for word in line.xpath('alto:String/@CONTENT', namespaces=NS))
        text_lines.append(text)  # Append each extracted text line to the list
    
    return " ".join(text_lines)  # Return all text as a single string

Erstellen einer neuen Spalte "text". In diese wird das jeweilige Ergebnis der Anwendung der Funktion "extract_text" auf das korrespondierende Element in der Spalte "content" des Dataframes "df" geschrieben: 

In [30]:
df['text'] = df['content'].apply(extract_text)
df

,filename,content,text
0,1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm...",LA OTRA ALEMANIA Registro Nacional de la Propi...
1,1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm...",ZUER NEW YORKER ERKLAERUNG DER OESTERREICHISCH...
2,1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm...","Steiermark, der sich durch besonders Quälereie..."
3,1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm...",ZUR DISKUSION UEBER SOZIALDEMOKRATISCHE In der...
4,1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm...",«exekutive zu fordern. Nur dadurch wäre die Si...
...,...,...,...
496,1943-04-01_M3RQ2MXZFORPDLNXQA57XR6C5MNN3EFS/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm...","j. J, Sansombre: DAUERKRISE DES DEUTSCHEN GENE..."
497,1943-04-01_M3RQ2MXZFORPDLNXQA57XR6C5MNN3EFS/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm...",Allgewalt ein und musste hinter den rieügewonn...
498,1943-04-01_M3RQ2MXZFORPDLNXQA57XR6C5MNN3EFS/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm...",gensatz zwischen Generalstab und politischen L...
499,1943-04-01_M3RQ2MXZFORPDLNXQA57XR6C5MNN3EFS/DD...,"<?xml version=""1.0"" encoding=""UTF-8""?><alto xm...",dements durch die alliierte Flugwaffe und die ...


Löschen der Spalte "content" um Speicherplatz zu sparen: 

In [31]:
df2 = df.drop(columns=['content'])
df2

,filename,text
0,1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DD...,LA OTRA ALEMANIA Registro Nacional de la Propi...
1,1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DD...,ZUER NEW YORKER ERKLAERUNG DER OESTERREICHISCH...
2,1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DD...,"Steiermark, der sich durch besonders Quälereie..."
3,1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DD...,ZUR DISKUSION UEBER SOZIALDEMOKRATISCHE In der...
4,1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DD...,«exekutive zu fordern. Nur dadurch wäre die Si...
...,...,...
496,1943-04-01_M3RQ2MXZFORPDLNXQA57XR6C5MNN3EFS/DD...,"j. J, Sansombre: DAUERKRISE DES DEUTSCHEN GENE..."
497,1943-04-01_M3RQ2MXZFORPDLNXQA57XR6C5MNN3EFS/DD...,Allgewalt ein und musste hinter den rieügewonn...
498,1943-04-01_M3RQ2MXZFORPDLNXQA57XR6C5MNN3EFS/DD...,gensatz zwischen Generalstab und politischen L...
499,1943-04-01_M3RQ2MXZFORPDLNXQA57XR6C5MNN3EFS/DD...,dements durch die alliierte Flugwaffe und die ...


Speichern als CSV: 

In [32]:
df2.to_csv("otra_alemania_content.csv", encoding = "UTF-8")